In [4]:
# general imports
from math import pi
import json
import time
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.devices import Devices

%matplotlib inline

In [5]:
device = AwsDevice(Devices.IQM.Emerald)

execution_windows = device.properties.service.executionWindows
connectivity_graph = device.properties.paradigm.connectivity
calibration = device.properties.standardized.dict()

print(f"The availability windows for {device.name}:\n{execution_windows}\n")
print(f"The connectivity graph of the qubits for this device:\n {connectivity_graph}\n")
# print("Calibration data:\n", json.dumps(calibration, sort_keys=True, indent=2))
with open('AWS_Qubit_Calibration.json', 'w', encoding='utf-8') as f:
    json.dump(calibration, f, indent=4) # 'indent=4' makes the file human-readable


The availability windows for Emerald:
[DeviceExecutionWindow(executionDay=<ExecutionDay.WEEKDAYS: 'Weekdays'>, windowStartHour=datetime.time(0, 0), windowEndHour=datetime.time(3, 29, 59)), DeviceExecutionWindow(executionDay=<ExecutionDay.WEEKDAYS: 'Weekdays'>, windowStartHour=datetime.time(7, 0), windowEndHour=datetime.time(16, 29, 59)), DeviceExecutionWindow(executionDay=<ExecutionDay.WEEKDAYS: 'Weekdays'>, windowStartHour=datetime.time(20, 0), windowEndHour=datetime.time(23, 59, 59))]

The connectivity graph of the qubits for this device:
 fullyConnected=False connectivityGraph={'1': ['2', '5'], '2': ['1', '6'], '5': ['1', '4', '6', '11'], '6': ['2', '5', '7', '12'], '3': ['4', '9'], '4': ['3', '5', '10'], '9': ['3', '8', '10'], '10': ['4', '9', '11'], '11': ['5', '10', '12', '19'], '7': ['6'], '12': ['6', '11', '13', '20'], '8': ['9', '16'], '16': ['8', '15', '17', '24'], '19': ['11', '18', '20', '27'], '13': ['12', '14', '21'], '20': ['12', '19', '21', '28'], '14': ['13', '22'], '2

In [6]:
import graphconnector as gc

G = device.topology_graph

start_time = time.time()
print("Initializing solver...")
total_connectivity = device.properties.paradigm.connectivity.connectivityGraph
qubits = [qub for qub in total_connectivity]
qubits.sort(key=float)

final_sets, solver = gc.generate_circuits(total_connectivity, max_sets=4, compaction = True, redundancy = True)

end_time = time.time()

print(f"\nDONE in {end_time - start_time:.4f} seconds.")
print(f"Total Edges in Graph: {len(solver.all_edges)}")
print("-" * 30)

for i in range(len(final_sets)):
    
    red_edges = list(final_sets[i])
    connected_qubits = [str(qub_pair[0]) for qub_pair in red_edges] + [str(qub_pair[1]) for qub_pair in red_edges]
    connected_qubits.sort(key=float)
    # print("Connected Qubits:\n",connected_qubits,"\n")
    # print("Total Qubits:\n",qubits,"\n")
    print("Efficiency:", len(connected_qubits)/len(qubits))
    
    # 3. Create a color map based on G.edges()
    edge_colors = []
    widths = []
    for u, v in G.edges():
        u = str(u)
        v = str(v)
        # Check if the edge (or its reverse) is in your red_subset
        if (u, v) in red_edges or (v, u) in red_edges:
            edge_colors.append('red')
            widths.append(6)
        else:
            edge_colors.append('black')
            widths.append(3)
    
    black_edges = [e for e in G.edges() if e not in red_edges and (e[1], e[0]) not in red_edges]

    if i == 0:
        pos = nx.forceatlas2_layout(G)
    # nx.draw(G, pos, with_labels=True, edge_color=edge_colors, node_color = 'black' ,font_color="white", width = widths, arrows=False)
    # plt.show()


Initializing solver...
Graph Degree: 4. Solving with max 5 circuits.
----------------------------------------
VERIFICATION REPORT
----------------------------------------
Total Number of Circuits:    4
Total Unique Edges in Graph: 79
Unique Edges Tested:         79
Total Tests Performed:       93 (Avg 1.18x redundancy)
Coverage:                    100.0%

[PASS] All connections tested at least once.
----------------------------------------

DONE in 0.0020 seconds.
Total Edges in Graph: 79
------------------------------
Efficiency: 0.9433962264150944
Efficiency: 0.9056603773584906
Efficiency: 0.7924528301886793
Efficiency: 0.8679245283018868


In [7]:
def Simple_Identity_Circuit(qubits = [1,2,3]):
    Simple_Identity = Circuit()
    for qub in qubits:
        qub_in = int(qub)
        Simple_Identity = Simple_Identity.prx(target = qub_in, angle_1 = pi, angle_2 = 0)
        Simple_Identity = Simple_Identity.prx(target = qub_in, angle_1 = pi, angle_2 = 0)
    return Simple_Identity

def Spin_Echo_Identity_Circuit(qubits = [1,2,3]):
    Spin_Echo_Identity = Circuit()
    for qub in qubits:
        qub_in = int(qub)
        Spin_Echo_Identity = Spin_Echo_Identity.prx(target = qub_in, angle_1 = pi/2, angle_2 = 0)
        Spin_Echo_Identity = Spin_Echo_Identity.prx(target = qub_in, angle_1 = pi, angle_2 = pi/2)
        Spin_Echo_Identity = Spin_Echo_Identity.prx(target = qub_in, angle_1 = pi/2, angle_2 = 0)
        Spin_Echo_Identity = Spin_Echo_Identity.prx(target = qub_in, angle_1 = pi, angle_2 = pi/2)
    return Spin_Echo_Identity

def Simple_Flip_Circuit(qubits = [1,2,3]):
    Simple_Flip = Circuit()
    for qub in qubits:
        qub_in = int(qub)
        Simple_Flip = Simple_Flip.prx(target = qub_in, angle_1 = pi/2, angle_2 = 0)
        Simple_Flip = Simple_Flip.prx(target = qub_in, angle_1 = pi/2, angle_2 = 0)
    return Simple_Flip

def Spin_Echo_Flip_Circuit(qubits = [1,2,3]):
    Spin_Echo_Flip = Circuit()
    for qub in qubits:
        qub_in = int(qub)
        Spin_Echo_Flip = Spin_Echo_Flip.prx(target = qub_in, angle_1 = pi/2, angle_2 = 0)
        Spin_Echo_Flip = Spin_Echo_Flip.prx(target = qub_in, angle_1 = pi, angle_2 = pi/2)
        Spin_Echo_Flip = Spin_Echo_Flip.prx(target = qub_in, angle_1 = -pi/2, angle_2 = 0)
        Spin_Echo_Flip = Spin_Echo_Flip.prx(target = qub_in, angle_1 = pi, angle_2 = pi/2)
    return Spin_Echo_Flip

In [8]:
total_connectivity = device.properties.paradigm.connectivity.connectivityGraph
qubits = [qub for qub in total_connectivity]
qubits.sort(key=float)

In [9]:
Verbatim_Simple_Identity_Circuit = Circuit().add_verbatim_box(Simple_Identity_Circuit(qubits))
Verbatim_Simple_Flip_Circuit = Circuit().add_verbatim_box(Simple_Flip_Circuit(qubits))
Verbatim_Spin_Echo_Identity_Circuit = Circuit().add_verbatim_box(Spin_Echo_Identity_Circuit(qubits))
Verbatim_Spin_Echo_Flip_Circuit = Circuit().add_verbatim_box(Spin_Echo_Flip_Circuit(qubits))

In [10]:
Verbatim_Simple_Identity_task = device.run(Verbatim_Simple_Identity_Circuit, shots=8192, disable_qubit_rewiring=True)

In [25]:
Verbatim_Simple_Identity_state = Verbatim_Simple_Identity_task.state()
Verbatim_Simple_Identity_state

'COMPLETED'

In [26]:
Verbatim_Simple_Identity_result = Verbatim_Simple_Identity_task.result()
Verbatim_Simple_Identity_measured_qubits = Verbatim_Simple_Identity_result.measured_qubits
Verbatim_Simple_Identity_measurements = Verbatim_Simple_Identity_result.measurements
print(np.array(Verbatim_Simple_Identity_measurements).size)


np.savetxt("/home/ec2-user/SageMaker/measurement-test/Verbatim_Simple_Identity_measured_qubits.csv", Verbatim_Simple_Identity_measured_qubits, delimiter=",")
np.savetxt("/home/ec2-user/SageMaker/measurement-test/Verbatim_Simple_Identity_measurements.csv", Verbatim_Simple_Identity_measurements, delimiter=",")


434176


In [ ]:
Verbatim_Simple_Flip_task = device.run(Verbatim_Simple_Flip_Circuit, shots=8192, disable_qubit_rewiring=True)

In [ ]:
Verbatim_Simple_Flip_result = Verbatim_Simple_Flip_task.result()
Verbatim_Simple_Flip_measured_qubits = Verbatim_Simple_Flip_result.measured_qubits
Verbatim_Simple_Flip_measurements = Verbatim_Simple_Flip_result.measurements
print(np.array(Verbatim_Simple_Flip_measurements).size)


np.savetxt("/home/ec2-user/SageMaker/measurement-test/Verbatim_Simple_Flip_measured_qubits.csv", Verbatim_Simple_Identity_measured_qubits, delimiter=",")
np.savetxt("/home/ec2-user/SageMaker/measurement-test/Verbatim_Simple_Flip_measurements.csv", Verbatim_Simple_Identity_measurements, delimiter=",")


In [ ]:
# Verbatim_Spin_Echo_Identity_task = device.run(Verbatim_Spin_Echo_Identity_Circuit, shots=8192, disable_qubit_rewiring=True)
Verbatim_Spin_Echo_Identity_result = Verbatim_Spin_Echo_Identity_task.result()
Verbatim_Spin_Echo_Identity_measured_qubits = Verbatim_Spin_Echo_Identity_result.measured_qubits
Verbatim_Spin_Echo_Identity_measurements = Verbatim_Spin_Echo_Identity_result.measurements
print(np.array(Verbatim_Spin_Echo_Identity_measurements).size)


np.savetxt("/home/ec2-user/SageMaker/measurement-test/Verbatim_Spin_Echo_Identity_measured_qubits.csv", Verbatim_Simple_Identity_measured_qubits, delimiter=",")
np.savetxt("/home/ec2-user/SageMaker/measurement-test/Verbatim_Spin_Echo_Identity_measurements.csv", Verbatim_Simple_Identity_measurements, delimiter=",")


In [ ]:
# Verbatim_Spin_Echo_Flip_task = device.run(Verbatim_Spin_Echo_Flip_Circuit, shots=8192, disable_qubit_rewiring=True)Verbatim_Simple_Identity_result = Verbatim_Simple_Identity_task.result()
Verbatim_Spin_Echo_Flip_result = Verbatim_Spin_Echo_Flip_task.result()
Verbatim_Spin_Echo_Flip_measured_qubits = Verbatim_Spin_Echo_Flip_result.measured_qubits
Verbatim_Spin_Echo_Flip_measurements = Verbatim_Spin_Echo_Flip_result.measurements
print(np.array(Verbatim_Spin_Echo_Flip_measurements).size)


np.savetxt("/home/ec2-user/SageMaker/measurement-test/Verbatim_Spin_Echo_Flip_measured_qubits.csv", Verbatim_Simple_Identity_measured_qubits, delimiter=",")
np.savetxt("/home/ec2-user/SageMaker/measurement-test/Verbatim_Spin_Echo_Flip_measurements.csv", Verbatim_Simple_Identity_measurements, delimiter=",")
